In [4]:
#Step 1: Install Necessary Packages
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import config

In [5]:
#Load API KEY
API_KEY = config.api_key
print("Key Loaded Succesfully")

Key Loaded Succesfully


In [ ]:
#Creating Parameters, and informing NASDAQ how many rows we wish to pull
api_url = 'https://data.nasdaq.com/api/v3/datatables/MER/F1.json'

parameters = {
    'api_key': API_KEY,
    'qopts.per_page': 2
}

In [7]:
print(parameters)

{'api_key': 'NvQ2e2AGypxf8sbq5BUH', 'qopts.per_page': 2}
